In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from scipy.spatial.distance import cosine

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [4]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [5]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')
df_movies

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10192,65088,Bedtime Stories,960731,Más allá de los sueños,http://ia.media-imdb.com/images/M/MV5BMjA5Njk5...,2008,bedtime_stories,4.4,104,26,...,25,4.7,26,6,20,23,3.5,108877,63,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,25464,El enemigo público número 1,http://ia.media-imdb.com/images/M/MV5BMTUyODE3...,1934,manhattan_melodrama,7,12,10,...,83,0,4,2,2,50,3.7,344,71,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008,choke,5.6,135,73,...,54,4.9,26,8,18,30,3.3,13893,55,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,959337,Revolutionary Road,http://ia.media-imdb.com/images/M/MV5BMTI2MzY2...,2008,revolutionary_road,6.7,194,133,...,68,6.9,36,25,11,69,3.5,46044,70,http://content8.flixster.com/movie/10/88/40/10...


Recode users ids

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [8]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [9]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [10]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

## Матрица рейтингов

In [11]:
from scipy.sparse import coo_matrix, csr_matrix

In [12]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [13]:
R
df_rates.movieID.values

array([   2,   31,  105, ..., 9211, 9900, 9957], dtype=int64)

## Похожесть между пользователями

In [14]:
R = R.tocsr()

In [23]:
def cosine_similarity_pair_users(u, v):
    ## Your code here
    idx =(u.toarray() != 0) & (v.toarray() != 0)
    v1 = v[idx]
    u1 =u[idx]
    if v1.shape[1] <=2 or u1.shape[1]<=2:
        return 0
    i = 0
    res = 0
    res1 = 0
    res2 = 0
    while True:
        try:
            res += v1[0,i] * u1[0,i]
            res1 += v1[0,i]**2
            res2 += u1[0,i]**2
            i+=1
        except IndexError:
            break     
    return res/(np.sqrt(res1)*np.sqrt(res2))
r = cosine_similarity_pair_users(R[42], R[1378])
print(r)

0


In [24]:
r1 = np.squeeze(np.asarray(R[146]))
r2 = np.squeeze(np.asarray(R[239]))
print(R.shape)
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

(2113, 10109)
0.923


## Функция нахождения пользователей, схожих с данным. 

In [25]:
def similar_users(u, R, n_neigbours):
    ## Your code here
    ousers = np.zeros((R.shape[0]))
    for i in range(2113):
        ousers[i] = cosine_similarity_pair_users(R[u], R[i])
    res = np.argsort(ousers)[::-1]
    return(res[:n_neigbours])
similar_users(42, R, 10)

array([  42,  281,  633,  724,  815,    2,  620,  650, 1692, 1506],
      dtype=int64)

In [26]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [27]:
answer2

'42281633724815262065016921506'

## Функция прогнозирования рейтинга

In [28]:
def rate_items_user(u, R, n_neigbours):
    predictions = np.zeros((R.shape[1]))
    neighbours = np.delete(similar_users(u, R, n_neigbours+1),1)
    sn = dict()
    for i in neighbours:
        sn[i] = cosine_similarity_pair_users(R[u], R[i])
    for j in range(R.shape[1]):
        if R[u,j] == 0:
            sinmodule = 0
            sin = 0
            for k in range(30):
                if R[neighbours[k],j]!=0:
                    sinmodule +=  abs(sn[neighbours[k]])
                    sin += R[neighbours[k],j]*sn[neighbours[k]]
            predictions[j] = sin/sinmodule
    res = np.argsort(predictions)
    print(predictions[res[-1]])
    print(predictions[res[-2]])
    print(predictions[res[-3]])
    print(predictions[res[-4]])
    return res[-5:]


#rate_items_user(20, R, n_neigbours=30)

In [115]:
R_hat = rate_items_user(20, R, n_neigbours=30)

ZeroDivisionError: division by zero